In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
dataset = pd.read_csv("/kaggle/input/coffee-shop-daily-revenue-prediction-dataset/coffee_shop_revenue.csv")
X = dataset[["Number_of_Customers_Per_Day","Average_Order_Value","Marketing_Spend_Per_Day"]]
X = np.array(X).T
X_max = np.max(X, axis=1, keepdims=True)
X_min = np.min(X,axis=1, keepdims=True)
X = (X - X_min) / (X_max - X_min)

In [ ]:
Y = dataset["Daily_Revenue"]
Y = np.array(Y).reshape((1,len(Y)))
Y_max = np.max(Y, axis=1,keepdims=True)
Y_min = np.min(Y, axis=1,keepdims=True)
Y = (Y - Y_min) / (Y_max - Y_min)


In [ ]:
def layers(X,y):
    n_x = X.shape[0]
    n_h = 10
    n_y = Y.shape[0]
    return (n_x,n_h,n_y)

In [ ]:
def init_parameters(n_x,n_h,n_y):
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    parameters = {"W1":W1,
                 "b1":b1,
                 "W2":W2,
                 "b2":b2}
    return parameters

In [ ]:
def relu(Z):
    return np.maximum(0,Z)
    

In [ ]:
def forward_propagation(X,parameters):
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]

    Z1 = W1@X+b1
    A1 = relu(Z1)
    Z2 = W2@A1+b2
    A2 = Z2

    cc = {"Z1":Z1,
         "Z2":Z2,
         "A2":A2,
         "A1":A1}
    return A2,cc

In [ ]:
def comute_cost(y,A2):
    m = y.shape[1]
    cost = np.sum((A2-y)**2)/m
    return cost
    

In [ ]:
def backward_propagation(cc,parameters,X,y):
    W2 = parameters["W2"]
    A1 = cc["A1"]
    A2 = cc["A2"]
    m = X.shape[1]

    dZ2 = A2-y
    dW2 = 1/m * np.dot(dZ2,A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T,dZ2)*(A1>0)
    dW1 = 1/m * (dZ1@X.T)
    db1 = 1/m * np.sum(dZ1,axis=1, keepdims=True)

    grads = {"dW2":dW2,
            "db1":db1,
            "dW1":dW1,
            "db2":db2}
    return grads
    

In [ ]:
def update_parameters(parameters,grads,learning_rate=0.1):
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]

    dW1 = grads["dW1"]
    dW2 = grads["dW2"]
    db1 = grads["db1"]
    db2 = grads["db2"]

    W1 = W1 - learning_rate * dW1
    W2 = W2 - learning_rate * dW2
    b1 = b1 - learning_rate * db1
    b2 = b2 - learning_rate * db2

    parameters = {"W1":W1,
                 "W2":W2,
                 "b1":b1,
                 "b2":b2}
    return parameters
    

In [ ]:
def nn(X,y,learning_rate=0.1,epochs=100):
    n_x = layers(X,y)[0]
    n_y = layers(X,y)[2]
    n_h = 32

    parameters = init_parameters(n_x,n_h,n_y)
    for i in range(epochs):
        A2,cc = forward_propagation(X,parameters)
        cost = comute_cost(y,A2)
        grads = backward_propagation(cc,parameters,X,y)
        parameters = update_parameters(parameters,grads,learning_rate)
        if i%2 == 0:
            print(f"Cost at {i} iteration ", cost)

    return parameters

In [ ]:
parameters = nn(X,Y,0.01,500)

In [ ]:
def predict(X_pred,parameters):
    A2,_ = forward_propagation(X_pred,parameters)
    return A2

In [ ]:
W1 = parameters["W1"]
W2 = parameters["W2"]
b1 = parameters["b1"]
b2 = parameters["b2"]

In [ ]:
x_pred = np.array([[264],[7.6],[318.08]])
prediction = predict(x_pred,parameters)
print(prediction)
precition_orig = prediction * (Y_max - Y_min) + Y_min
print(precition_orig)